<a href="https://colab.research.google.com/github/ocr-workspace/Ocr-Model-testing-on-Retail-Structured-Invoice-/blob/main/Structured_Invoice_Vast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependency install

In [ ]:
import os
# This hides the second GPU, preventing the DataParallel crash!
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!pip install transformers==4.49.0

In [ ]:
!apt-get install tesseract-ocr -y
!pip install pytesseract
!pip install datasets
!pip install evaluate
!pip install opencv-python
!pip install pillow
!pip install pandas

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00


Loading dataset named Cord from hugging face

**Clustering Module** for table reconstruction

In [ ]:
from datasets import load_dataset

ds_invoice = load_dataset("naver-clova-ix/cord-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN

def cluster_rows(words, image_height):
    """
    words: list of dicts with keys:
           text, x_center, y_center, bbox
    image_height: height of invoice image

    Returns:
        list of row clusters (each cluster = list of words)
    """

    if len(words) == 0:
        return []

    # Extract Y centers
    y_coords = np.array([[w["y_center"]] for w in words])

    # Dynamic eps based on image height
    eps = image_height * 0.015  # 1.5% of height (tunable)

    clustering = DBSCAN(eps=eps, min_samples=1).fit(y_coords)

    labels = clustering.labels_

    row_clusters = {}

    for label, word in zip(labels, words):
        if label not in row_clusters:
            row_clusters[label] = []
        row_clusters[label].append(word)

    # Sort rows top-to-bottom
    sorted_rows = sorted(
        row_clusters.values(),
        key=lambda cluster: np.mean([w["y_center"] for w in cluster])
    )

    return sorted_rows

**Word Standardization Layer**

In [ ]:
def standardize_ocr_output(ocr_result):

    words = []

    for item in ocr_result:
        x_min, y_min, x_max, y_max = item["bbox"]

        words.append({
            "text": item["text"],
            "x_center": (x_min + x_max) / 2,
            "y_center": (y_min + y_max) / 2,
            "bbox": [x_min, y_min, x_max, y_max]
        })

    return words

**Tesseract Wrapper**

In [ ]:
import pytesseract
import cv2
import numpy as np

def tesseract_ocr(image):

    # Convert to RGB if needed
    if isinstance(image, np.ndarray):
        img = image
    else:
        img = np.array(image)

    data = pytesseract.image_to_data(
        img,
        output_type=pytesseract.Output.DICT
    )

    words = []

    n_boxes = len(data['text'])

    for i in range(n_boxes):

        text = data['text'][i].strip()

        if text == "":
            continue

        x = data['left'][i]
        y = data['top'][i]
        w = data['width'][i]
        h = data['height'][i]

        words.append({
            "text": text,
            "bbox": [x, y, x+w, y+h]
        })

    return words

**EasyOCR Wrapper**

In [ ]:
!pip install easyocr

In [ ]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True)

def easyocr_ocr(image):

    if not isinstance(image, np.ndarray):
        image = np.array(image)

    results = reader.readtext(image)

    words = []

    for (bbox, text, conf) in results:

        x_coords = [pt[0] for pt in bbox]
        y_coords = [pt[1] for pt in bbox]

        x_min = int(min(x_coords))
        y_min = int(min(y_coords))
        x_max = int(max(x_coords))
        y_max = int(max(y_coords))

        words.append({
            "text": text,
            "bbox": [x_min, y_min, x_max, y_max]
        })

    return words

**PaddleOCR Wrapper**

In [ ]:
!pip install paddlepaddle-gpu


In [ ]:
# Force-uninstall the broken numpy
!pip uninstall numpy -y

# Install the packages but strictly lock numpy at 1.26.4 simultaneously
!pip install numpy==1.26.4 paddleocr==2.8.1 evaluate doctr

In [ ]:
!pip list | grep -E "numpy|paddle|evaluate"

evaluate                                 0.4.6
numpy                                    1.26.4
paddleocr                                2.8.1
paddlepaddle-gpu                         2.6.2


In [ ]:
# Uninstall langchain to bypass the PaddleX import bug
# This will NOT touch your numpy or paddlepaddle installations!
!pip uninstall langchain langchain-community langchain-core -y

In [ ]:
import numpy as np

# 1. The Monkey Patch: Recreate the missing np.sctypes dictionary for imgaug
if not hasattr(np, 'sctypes'):
    np.sctypes = {
        'int': [np.int8, np.int16, np.int32, np.int64],
        'uint': [np.uint8, np.uint16, np.uint32, np.uint64],
        'float': [np.float16, np.float32, np.float64],
        'complex': [np.complex64, np.complex128],
        'others': [bool, object, bytes, str, np.void]
    }

# 2. Now import PaddleOCR (it will load successfully because the crash is bypassed!)
from paddleocr import PaddleOCR

# 3. Initialize your model
paddle_model = PaddleOCR(
    use_angle_cls=True,
    lang='en',
    use_gpu=True
)

print("PaddleOCR successfully loaded!")

def paddle_ocr(image):

    if not isinstance(image, np.ndarray):
        image = np.array(image)

    result = paddle_model.ocr(image, cls=True)

    words = []

    for line in result:
        for word_info in line:

            box = word_info[0]
            text = word_info[1][0]

            x_coords = [pt[0] for pt in box]
            y_coords = [pt[1] for pt in box]

            x_min = int(min(x_coords))
            y_min = int(min(y_coords))
            x_max = int(max(x_coords))
            y_max = int(max(y_coords))

            words.append({
                "text": text,
                "bbox": [x_min, y_min, x_max, y_max]
            })

    return words

**DocTR Wrapper**

In [ ]:
!pip install python-doctr[torch]

In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

doctr_model = ocr_predictor(pretrained=True)

def doctr_ocr(image):

    if not isinstance(image, np.ndarray):
        image = np.array(image)

    result = doctr_model([image])

    words = []

    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:

                    x_min, y_min = word.geometry[0]
                    x_max, y_max = word.geometry[1]

                    h, w = image.shape[:2]

                    words.append({
                        "text": word.value,
                        "bbox": [
                            int(x_min * w),
                            int(y_min * h),
                            int(x_max * w),
                            int(y_max * h)
                        ]
                    })

    return words

  0%|          | 0/65814772 [00:00<?, ?it/s]

  0%|          | 0/63303144 [00:00<?, ?it/s]

**Ground Truth Text Builder**

In [ ]:
import json

def build_gt_text(sample):

    gt_outer = json.loads(sample["ground_truth"])
    parsed = gt_outer["gt_parse"]

    lines = []

    menu = parsed.get("menu", [])

    # Case 1: menu is list (normal case)
    if isinstance(menu, list):
        for item in menu:
            if isinstance(item, dict):
                cnt = item.get("cnt", "")
                name = item.get("nm", "")
                price = item.get("price", "")
                lines.append(f"{cnt} {name} {price}")

    # Case 2: menu is single dict (edge case)
    elif isinstance(menu, dict):
        cnt = menu.get("cnt", "")
        name = menu.get("nm", "")
        price = menu.get("price", "")
        lines.append(f"{cnt} {name} {price}")

    # Subtotal
    sub = parsed.get("sub_total", {})
    if isinstance(sub, dict):
        for value in sub.values():
            lines.append(str(value))

    # Total
    total = parsed.get("total", {})
    if isinstance(total, dict) and "total_price" in total:
        lines.append(str(total["total_price"]))

    return " ".join(lines)

**Universal Evalaution Runner**

In [ ]:
!pip install jiwer

In [ ]:
import evaluate
import time
import pandas as pd

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

def run_ocr_benchmark(ds, ocr_function, model_name, samples=50):

    results = []

    for i in range(samples):

        sample = ds["train"][i]
        image = sample["image"]
        gt_text = build_gt_text(sample)

        start_time = time.time()

        words = ocr_function(image)

        if isinstance(words, str):
           ocr_text = words
        else:
           ocr_text = " ".join([w["text"] for w in words])

        end_time = time.time()

        cer = cer_metric.compute(
            predictions=[ocr_text],
            references=[gt_text]
        )

        wer = wer_metric.compute(
            predictions=[ocr_text],
            references=[gt_text]
        )

        results.append({
            "CER": cer,
            "WER": wer,
            "Time": end_time - start_time
        })

    df = pd.DataFrame(results)

    print(f"\n===== {model_name} Results =====")
    print("Mean CER:", df["CER"].mean())
    print("Mean WER:", df["WER"].mean())
    print("Avg Time:", df["Time"].mean())

    return df

**Tesseract**

In [ ]:
for i in range(5):
    print(type(json.loads(ds_invoice["train"][i]["ground_truth"])["gt_parse"]["menu"]))

In [ ]:
df_tess = run_ocr_benchmark(ds_invoice, tesseract_ocr, "Tesseract")


===== Tesseract Results =====
Mean CER: 1.2425900779068786
Mean WER: 1.663429996573475
Avg Time: 0.9068947362899781


**EasyOCR**

In [ ]:
df_easy = run_ocr_benchmark(ds_invoice, easyocr_ocr, "EasyOCR")


===== EasyOCR Results =====
Mean CER: 1.1637943945368556
Mean WER: 1.7482964184519856
Avg Time: 0.5298595905303956


In [ ]:
df_paddle = run_ocr_benchmark(ds_invoice, paddle_ocr, "PaddleOCR")

[2026/02/24 12:37:59] ppocr DEBUG: dt_boxes num : 80, elapsed : 0.8221826553344727
[2026/02/24 12:37:59] ppocr DEBUG: cls num  : 80, elapsed : 0.2342057228088379
[2026/02/24 12:38:00] ppocr DEBUG: rec_res num  : 80, elapsed : 0.31259799003601074
[2026/02/24 12:38:00] ppocr DEBUG: dt_boxes num : 29, elapsed : 0.04682731628417969
[2026/02/24 12:38:00] ppocr DEBUG: cls num  : 29, elapsed : 0.08996272087097168
[2026/02/24 12:38:00] ppocr DEBUG: rec_res num  : 29, elapsed : 0.1291031837463379
[2026/02/24 12:38:00] ppocr DEBUG: dt_boxes num : 24, elapsed : 0.0569760799407959
[2026/02/24 12:38:00] ppocr DEBUG: cls num  : 24, elapsed : 0.06496500968933105
[2026/02/24 12:38:00] ppocr DEBUG: rec_res num  : 24, elapsed : 0.05875110626220703
[2026/02/24 12:38:00] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.06055402755737305
[2026/02/24 12:38:00] ppocr DEBUG: cls num  : 20, elapsed : 0.03250241279602051
[2026/02/24 12:38:00] ppocr DEBUG: rec_res num  : 20, elapsed : 0.05735635757446289
[2026/02/24 

**PaddleOcr**

**Doctr**

In [ ]:
df_doctr = run_ocr_benchmark(ds_invoice, doctr_ocr, "DocTR")


===== DocTR Results =====
Mean CER: 1.193762271922286
Mean WER: 1.407587275966102
Avg Time: 2.0541904258728025


**Paddle Ocr(detection) with TrOcr(recognition)**

In [ ]:
!pip install transformers timm sentencepiece -q

In [ ]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = TrOCRProcessor.from_pretrained(
    "microsoft/trocr-base-printed"
)

model_trocr = VisionEncoderDecoderModel.from_pretrained(
    "microsoft/trocr-base-printed"
).to(device)

model_trocr.eval()

In [ ]:
from paddleocr import PaddleOCR

ocr_det = PaddleOCR(
    use_angle_cls=True,
    lang="en",
    rec=False,         # ❗ Detection only
    use_gpu=True
)

**Hybrid OCR Function**

In [ ]:
import numpy as np
import cv2

def hybrid_ocr(image):

    if not isinstance(image, np.ndarray):
        image = np.array(image)

    result = ocr_det.ocr(image, cls=True)

    if result is None:
        return ""

    boxes = []
    for line in result:
        for word_info in line:
            boxes.append(word_info[0])

    # Sort top-to-bottom, left-to-right
    boxes = sorted(boxes, key=lambda x: (x[0][1], x[0][0]))

    recognized_words = []

    for box in boxes:

        pts = np.array(box).astype(int)

        x_min = max(0, min(pts[:,0]))
        y_min = max(0, min(pts[:,1]))
        x_max = min(image.shape[1], max(pts[:,0]))
        y_max = min(image.shape[0], max(pts[:,1]))

        crop = image[y_min:y_max, x_min:x_max]

        if crop.size == 0:
            continue

        # Resize for better transformer recognition
        crop = cv2.resize(crop, None, fx=2, fy=2)

        pixel_values = processor(
            images=crop,
            return_tensors="pt"
        ).pixel_values.to(device)

        with torch.no_grad():
            generated_ids = model_trocr.generate(pixel_values)

        text = processor.batch_decode(
            generated_ids,
            skip_special_tokens=True
        )[0]

        recognized_words.append(text)

    return " ".join(recognized_words)

In [ ]:
df_hybrid = run_ocr_benchmark(ds_invoice, hybrid_ocr, "Hybrid")

[2026/02/24 12:54:54] ppocr DEBUG: dt_boxes num : 80, elapsed : 0.06696462631225586
[2026/02/24 12:54:54] ppocr DEBUG: cls num  : 80, elapsed : 0.15387940406799316
[2026/02/24 12:54:54] ppocr DEBUG: rec_res num  : 80, elapsed : 0.2423720359802246
[2026/02/24 12:55:02] ppocr DEBUG: dt_boxes num : 29, elapsed : 0.04482865333557129
[2026/02/24 12:55:02] ppocr DEBUG: cls num  : 29, elapsed : 0.04314899444580078
[2026/02/24 12:55:02] ppocr DEBUG: rec_res num  : 29, elapsed : 0.07915234565734863
[2026/02/24 12:55:05] ppocr DEBUG: dt_boxes num : 24, elapsed : 0.04288029670715332
[2026/02/24 12:55:05] ppocr DEBUG: cls num  : 24, elapsed : 0.06281566619873047
[2026/02/24 12:55:05] ppocr DEBUG: rec_res num  : 24, elapsed : 0.05519843101501465
[2026/02/24 12:55:08] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.04532909393310547
[2026/02/24 12:55:08] ppocr DEBUG: cls num  : 20, elapsed : 0.034276723861694336
[2026/02/24 12:55:08] ppocr DEBUG: rec_res num  : 20, elapsed : 0.0469670295715332
[2026/02/

**Donut**

In [ ]:
!pip install transformers sentencepiece timm -q

Loading model

In [ ]:
import torch
from transformers import DonutProcessor, VisionEncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = DonutProcessor.from_pretrained(
    "naver-clova-ix/donut-base-finetuned-cord-v2"
)

model_donut = VisionEncoderDecoderModel.from_pretrained(
    "naver-clova-ix/donut-base-finetuned-cord-v2"
).to(device)

model_donut.eval()

Donut Ocr function that returns structured json

In [ ]:
import json

def donut_inference(image):

    if image.mode != "RGB":
        image = image.convert("RGB")

    # Prepare image
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

    # Prompt for CORD task
    task_prompt = "<s_cord-v2>"
    decoder_input_ids = processor.tokenizer(
        task_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    ).input_ids.to(device)

    with torch.no_grad():
        outputs = model_donut.generate(
            pixel_values,
            decoder_input_ids=decoder_input_ids,
            max_length=512,
            early_stopping=True,
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
            use_cache=True,
        )

    seq = processor.batch_decode(outputs, skip_special_tokens=True)[0]

    # Convert to JSON
    try:
        result = processor.token2json(seq)
    except:
        result = {}

    return result

Evalaution function for Donut

In [ ]:
import json

def evaluate_donut(ds, samples=50):

    total_correct = 0
    item_count_correct = 0

    for i in range(samples):

        sample = ds["train"][i]
        image = sample["image"]

        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        pred = donut_inference(image)

        # ---- FIXED HERE ----
        pred_parse = pred

        # ---- Total Accuracy ----
        gt_total = gt_parse.get("total", {}).get("total_price", "")
        pred_total = pred_parse.get("total", {}).get("total_price", "")

        total_match = int(gt_total == pred_total)
        total_correct += total_match

        # ---- Item Count Accuracy ----
        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))

        item_match = int(gt_items == pred_items)
        item_count_correct += item_match

    print("Total Accuracy %:", total_correct/samples*100)
    print("Item Count Accuracy %:", item_count_correct/samples*100)

In [ ]:
evaluate_donut(ds_invoice, samples=50)

Total Accuracy %: 100.0
Item Count Accuracy %: 100.0


**Testing robustness of Donut at diffrent noisy image**

Image Perturbation Function

In [ ]:
import numpy as np
import cv2
from PIL import Image

def perturb_image(image, mode="noise"):

    img = np.array(image)

    if mode == "noise":
        noise = np.random.normal(0, 25, img.shape).astype(np.uint8)
        img = cv2.add(img, noise)

    elif mode == "blur":
        img = cv2.GaussianBlur(img, (7,7), 0)

    elif mode == "downscale":
        h, w = img.shape[:2]
        img = cv2.resize(img, (w//2, h//2))
        img = cv2.resize(img, (w, h))

    elif mode == "rotate":
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), 5, 1)
        img = cv2.warpAffine(img, M, (w, h))

    elif mode == "brightness":
        img = cv2.convertScaleAbs(img, alpha=1.2, beta=40)

    return Image.fromarray(img)

> Robustness Evaluation

In [ ]:
def evaluate_donut_robust(ds, mode="noise", samples=50):

    total_correct = 0
    item_count_correct = 0

    for i in range(samples):

        sample = ds["train"][i]
        image = sample["image"]

        # Apply perturbation
        image = perturb_image(image, mode)

        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        pred = donut_inference(image)
        pred_parse = pred

        gt_total = gt_parse.get("total", {}).get("total_price", "")
        pred_total = pred_parse.get("total", {}).get("total_price", "")

        total_correct += int(gt_total == pred_total)

        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))

        item_count_correct += int(gt_items == pred_items)

    print(f"=== Mode: {mode} ===")
    print("Total Accuracy %:", total_correct/samples*100)
    print("Item Count Accuracy %:", item_count_correct/samples*100)

In [ ]:
evaluate_donut_robust(ds_invoice, mode="noise")
evaluate_donut_robust(ds_invoice, mode="blur")
evaluate_donut_robust(ds_invoice, mode="downscale")
evaluate_donut_robust(ds_invoice, mode="rotate")
evaluate_donut_robust(ds_invoice, mode="brightness")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== Mode: noise ===
Total Accuracy %: 34.0
Item Count Accuracy %: 40.0
=== Mode: blur ===
Total Accuracy %: 88.0
Item Count Accuracy %: 88.0
=== Mode: downscale ===
Total Accuracy %: 86.0
Item Count Accuracy %: 94.0
=== Mode: rotate ===
Total Accuracy %: 100.0
Item Count Accuracy %: 100.0
=== Mode: brightness ===
Total Accuracy %: 98.0
Item Count Accuracy %: 98.0


**Reconstruct text and then we check accuracy on diffrent matrix**

Reconstruction Function#

In [ ]:
import re

def reconstruct_invoice_from_text(text):

    lines = text.split("\n")
    menu = []
    total_price = ""

    for line in lines:

        # Try menu row
        match = re.search(r'(\d+)\s*x\s*(.*?)\s+([\d,.-]+)', line)
        if match:
            cnt = match.group(1) + " x"
            name = match.group(2).strip()
            price = match.group(3)
            menu.append({
                "cnt": cnt,
                "nm": name,
                "price": price
            })

        # Try total detection
        if "total" in line.lower():
            price_match = re.search(r'([\d,.-]+)', line)
            if price_match:
                total_price = price_match.group(1)

    return {
        "menu": menu,
        "total": {"total_price": total_price}
    }

* Structured Evaluation for Any OCR

In [ ]:
def evaluate_reconstruction(ds, ocr_function, samples=50):

    total_correct = 0
    item_count_correct = 0

    for i in range(samples):

        sample = ds["train"][i]
        image = sample["image"]

        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        raw_output = ocr_function(image)

        # ---- UNIVERSAL NORMALIZATION ----
        if isinstance(raw_output, str):
            ocr_text = raw_output

        elif isinstance(raw_output, list):
            # case: list of dicts
            if len(raw_output) > 0 and isinstance(raw_output[0], dict):
                ocr_text = " ".join([w.get("text", "") for w in raw_output])
            else:
                ocr_text = " ".join([str(w) for w in raw_output])

        else:
            ocr_text = str(raw_output)

        pred_parse = reconstruct_invoice_from_text(ocr_text)

        # ---- Total Accuracy ----
        gt_total = gt_parse.get("total", {}).get("total_price", "")
        pred_total = pred_parse.get("total", {}).get("total_price", "")

        total_correct += int(gt_total == pred_total)

        # ---- Item Count Accuracy ----
        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))

        item_count_correct += int(gt_items == pred_items)

    print("Total Accuracy %:", total_correct/samples*100)
    print("Item Count Accuracy %:", item_count_correct/samples*100)

In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

8

In [ ]:
import json

In [ ]:
evaluate_reconstruction(ds_invoice, paddle_ocr)


[2026/02/25 04:29:54] ppocr DEBUG: dt_boxes num : 80, elapsed : 1.3662643432617188
[2026/02/25 04:29:55] ppocr DEBUG: cls num  : 80, elapsed : 0.22235393524169922
[2026/02/25 04:29:55] ppocr DEBUG: rec_res num  : 80, elapsed : 0.32889890670776367
[2026/02/25 04:29:55] ppocr DEBUG: dt_boxes num : 29, elapsed : 0.04947257041931152
[2026/02/25 04:29:55] ppocr DEBUG: cls num  : 29, elapsed : 0.09712028503417969
[2026/02/25 04:29:55] ppocr DEBUG: rec_res num  : 29, elapsed : 0.13600683212280273
[2026/02/25 04:29:55] ppocr DEBUG: dt_boxes num : 24, elapsed : 0.05992603302001953
[2026/02/25 04:29:56] ppocr DEBUG: cls num  : 24, elapsed : 0.06924867630004883
[2026/02/25 04:29:56] ppocr DEBUG: rec_res num  : 24, elapsed : 0.0584104061126709
[2026/02/25 04:29:56] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.06280922889709473
[2026/02/25 04:29:56] ppocr DEBUG: cls num  : 20, elapsed : 0.034729957580566406
[2026/02/25 04:29:56] ppocr DEBUG: rec_res num  : 20, elapsed : 0.057021379470825195
[2026/02

In [ ]:
evaluate_reconstruction(ds_invoice, tesseract_ocr)

Total Accuracy %: 4.0
Item Count Accuracy %: 0.0


In [ ]:
evaluate_reconstruction(ds_invoice, easyocr_ocr)

Total Accuracy %: 2.0
Item Count Accuracy %: 0.0


In [ ]:
evaluate_reconstruction(ds_invoice, doctr_ocr)

Total Accuracy %: 2.0
Item Count Accuracy %: 0.0


In [ ]:


evaluate_reconstruction(ds_invoice, hybrid_ocr)

[2026/02/25 05:00:49] ppocr DEBUG: dt_boxes num : 80, elapsed : 0.06450271606445312
[2026/02/25 05:00:49] ppocr DEBUG: cls num  : 80, elapsed : 0.17209267616271973
[2026/02/25 05:00:49] ppocr DEBUG: rec_res num  : 80, elapsed : 0.25741004943847656
[2026/02/25 05:00:58] ppocr DEBUG: dt_boxes num : 29, elapsed : 0.04198718070983887
[2026/02/25 05:00:58] ppocr DEBUG: cls num  : 29, elapsed : 0.03947925567626953
[2026/02/25 05:00:58] ppocr DEBUG: rec_res num  : 29, elapsed : 0.0767512321472168
[2026/02/25 05:01:01] ppocr DEBUG: dt_boxes num : 24, elapsed : 0.03599214553833008
[2026/02/25 05:01:01] ppocr DEBUG: cls num  : 24, elapsed : 0.05639934539794922
[2026/02/25 05:01:01] ppocr DEBUG: rec_res num  : 24, elapsed : 0.054166316986083984
[2026/02/25 05:01:04] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.040987491607666016
[2026/02/25 05:01:04] ppocr DEBUG: cls num  : 20, elapsed : 0.032521724700927734
[2026/02/25 05:01:04] ppocr DEBUG: rec_res num  : 20, elapsed : 0.046328067779541016
[2026

**Function For getting structured output form paddleOcr**

In [ ]:
def paddle_ocr_structured(image):
    # Fix: Convert the PIL Image from the dataset into a NumPy array
    if not isinstance(image, np.ndarray):
        image = np.array(image)

    result = paddle_model.ocr(image, cls=True)

    words = []

    for line in result:
        for word_info in line:
            box = word_info[0]
            text = word_info[1][0]

            x_min = min([pt[0] for pt in box])
            y_min = min([pt[1] for pt in box])

            words.append({
                "text": text,
                "x": x_min,
                "y": y_min
            })

    return words

**Row Clustering **

In [ ]:
#clustring using y cordinate proximity
def cluster_rows(words):

    words = sorted(words, key=lambda w: w["y"])

    rows = []
    current_row = []

    if not words:
        return rows

    # Estimate average vertical spacing
    ys = sorted([w["y"] for w in words])
    avg_gap = sum(abs(ys[i] - ys[i-1]) for i in range(1, len(ys))) / max(1, len(ys)-1)

    y_threshold = avg_gap * 1.2  # adaptive

    for word in words:

        if not current_row:
            current_row.append(word)
            continue

        if abs(word["y"] - current_row[-1]["y"]) < y_threshold:
            current_row.append(word)
        else:
            rows.append(current_row)
            current_row = [word]

    if current_row:
        rows.append(current_row)

    return rows

**Reconstruction Function From  Spatial **

In [ ]:
def reconstruct_from_spatial(words):

    if not words:
        return {"menu": [], "total": {"total_price": ""}}

    rows = cluster_rows(words)

    menu = []
    total_price = ""

    # Collect all X positions
    x_positions = sorted([w["x"] for w in words])
    max_x = max(x_positions)

    # Estimate price column dynamically (rightmost 20%)
    price_threshold = max_x * 0.6   # more relaxed

    for row in rows:

        row = sorted(row, key=lambda w: w["x"])

        qty = ""
        price = ""
        name_tokens = []

        for w in row:
            text = w["text"]

            # Detect quantity (digit near left)
            if text.isdigit() and w["x"] < max_x * 0.2:
                qty = text

            # Detect price (number containing comma or digit cluster near right)
            elif any(c.isdigit() for c in text) and w["x"] > price_threshold:
                price = text

            else:
                name_tokens.append(text)

        line_text = " ".join([w["text"] for w in row]).lower()

        # Detect total row
        if "total" in line_text and price:
            total_price = price
            continue

        # Detect menu row
        if price and name_tokens:
            if not qty:
                qty = "1"
            menu.append({
                "cnt": qty + " x",
                "nm": " ".join(name_tokens),
                "price": price
            })

    return {
        "menu": menu,
        "total": {"total_price": total_price}
    }

**Evalaution function for spatial construction from Qwen LLm**

In [ ]:
def evaluate_spatial_paddle(ds, samples=50):

    total_correct = 0
    item_count_correct = 0

    for i in range(samples):

        sample = ds["train"][i]
        image = sample["image"]

        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        words = paddle_ocr_structured(image)
        pred_parse = reconstruct_from_spatial(words)

        gt_total = gt_parse.get("total", {}).get("total_price", "")
        pred_total = pred_parse.get("total", {}).get("total_price", "")

        total_correct += int(gt_total == pred_total)

        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))

        item_count_correct += int(gt_items == pred_items)

    print("Total Accuracy %:", total_correct/samples*100)
    print("Item Count Accuracy %:", item_count_correct/samples*100)

In [ ]:
evaluate_spatial_paddle(ds_invoice, samples=50)

[2026/02/25 06:10:08] ppocr DEBUG: dt_boxes num : 80, elapsed : 0.05966067314147949
[2026/02/25 06:10:09] ppocr DEBUG: cls num  : 80, elapsed : 0.11289453506469727
[2026/02/25 06:10:09] ppocr DEBUG: rec_res num  : 80, elapsed : 0.23305630683898926
[2026/02/25 06:10:09] ppocr DEBUG: dt_boxes num : 29, elapsed : 0.03991222381591797
[2026/02/25 06:10:09] ppocr DEBUG: cls num  : 29, elapsed : 0.04390525817871094
[2026/02/25 06:10:09] ppocr DEBUG: rec_res num  : 29, elapsed : 0.07392764091491699
[2026/02/25 06:10:09] ppocr DEBUG: dt_boxes num : 24, elapsed : 0.03527402877807617
[2026/02/25 06:10:09] ppocr DEBUG: cls num  : 24, elapsed : 0.03273439407348633
[2026/02/25 06:10:09] ppocr DEBUG: rec_res num  : 24, elapsed : 0.05725502967834473
[2026/02/25 06:10:09] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.0383145809173584
[2026/02/25 06:10:09] ppocr DEBUG: cls num  : 20, elapsed : 0.03424429893493652
[2026/02/25 06:10:09] ppocr DEBUG: rec_res num  : 20, elapsed : 0.049867868423461914
[2026/02

** Florence-2 Finetunning **

In [ ]:
import os
import json
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoProcessor,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

# ---------------------------------------------------
# Optional: helps debugging CUDA crashes
# Remove after everything works
# ---------------------------------------------------
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print("1. Initializing Processor...")

processor = AutoProcessor.from_pretrained(
    "microsoft/Florence-2-base",
    trust_remote_code=True
)

# ---------------------------------------------------
# DATASET
# ---------------------------------------------------
class RobustInvoiceDataset(Dataset):
    def __init__(self, hf_dataset, processor, split="train"):
        self.dataset = hf_dataset[split]
        self.processor = processor

        # IMPORTANT:
        # Florence expects task tokens.
        # Do NOT use free-form natural text here.
        self.task_prompt = "<ocr>"

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        image = sample["image"].convert("RGB")

        gt_json = json.loads(sample["ground_truth"])

        target_text = json.dumps({
            "menu": gt_json["gt_parse"].get("menu", []),
            "total": gt_json["gt_parse"].get("total", {})
        })

        # ---------------------------------------------------
        # DO NOT force padding or truncation here
        # Let collate_fn handle padding dynamically
        # ---------------------------------------------------
        inputs = self.processor(
            text=self.task_prompt,
            images=image,
            return_tensors="pt"
        )

        labels = processor.tokenizer(
            target_text,
            return_tensors="pt"
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "labels": labels["input_ids"].squeeze(0)
        }


train_dataset = RobustInvoiceDataset(ds_invoice, processor, split="train")
print(f"Factory initialized! Ready to process {len(train_dataset)} invoices.")

# ---------------------------------------------------
# MODEL
# ---------------------------------------------------
print("2. Loading the Florence-2 Brain...")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Florence-2-base",
    trust_remote_code=True
)

# DO NOT resize embeddings
# DO NOT modify vocab
# Florence already matches tokenizer internally

# ---------------------------------------------------
# Sanity Checks
# ---------------------------------------------------
print("Tokenizer vocab size:", processor.tokenizer.vocab_size)
print("Model vocab size:", model.config.vocab_size)

# ---------------------------------------------------
# COLLATE FUNCTION (Dynamic Padding)
# ---------------------------------------------------
def custom_collate_fn(batch):

    input_ids = torch.nn.utils.rnn.pad_sequence(
        [item["input_ids"] for item in batch],
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )

    attention_mask = torch.nn.utils.rnn.pad_sequence(
        [item["attention_mask"] for item in batch],
        batch_first=True,
        padding_value=0
    )

    pixel_values = torch.stack([item["pixel_values"] for item in batch])

    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch],
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )

    # Ignore padding tokens in loss
    labels[labels == processor.tokenizer.pad_token_id] = -100

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels
    }


print("3. Configuring the Kaggle Cockpit...")

training_args = TrainingArguments(
    output_dir="/kaggle/working/florence2-invoice-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=False,  # REQUIRED for multimodal models
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=custom_collate_fn,
)

print("\n🚀 IGNITION: Starting the fine-tuning process!")
trainer.train()

print("\nTraining Complete! Saving final weights...")
trainer.save_model("/kaggle/working/best_florence2_model")
print("Model saved successfully. Pipeline finished.")

1. Initializing Processor...
Factory initialized! Ready to process 800 invoices.
2. Loading the Florence-2 Brain...
Tokenizer vocab size: 50265
Model vocab size: 51289
3. Configuring the Kaggle Cockpit...

🚀 IGNITION: Starting the fine-tuning process!


Step,Training Loss
10,3.104900
20,1.478500
30,0.840800
40,0.646000
50,0.435100
60,0.444100
70,0.351500
80,0.366500
90,0.261900
100,0.257700



Training Complete! Saving final weights...
Model saved successfully. Pipeline finished.


In [ ]:
import json
import torch
import difflib
import re
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForCausalLM

# ==========================================
# 1. SETUP & HELPER FUNCTIONS
# ==========================================

print("1. Loading the Custom Brain...")
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)
model_path = "/kaggle/working/best_florence2_model"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# The exact list of coordinate/metadata keys we MUST ignore
IGNORE_KEYS = {
    'version', 'split', 'image_id', 'width', 'height',
    'x1', 'x2', 'x3', 'x4', 'y1', 'y2', 'y3', 'y4',
    'is_key', 'row_id', 'text', 'category', 'group_id', 'sub_group_id'
}

def get_similarity(str1, str2):
    """Returns a similarity score between 0 and 1 for two strings."""
    return difflib.SequenceMatcher(None, str(str1).lower(), str(str2).lower()).ratio()

def flatten_ground_truth(data):
    """
    Recursively extracts all string key-value pairs from nested ground truth,
    while explicitly filtering out OCR bounding boxes and metadata.
    """
    items = []
    if isinstance(data, dict):
        for k, v in data.items():
            if k in IGNORE_KEYS:
                continue # Skip the junk data!

            if isinstance(v, (dict, list)):
                items.extend(flatten_ground_truth(v))
            else:
                items.append((str(k).strip(), str(v).strip()))
    elif isinstance(data, list):
        for item in data:
            items.extend(flatten_ground_truth(item))
    return items

def salvage_predictions(text):
    """
    Uses regex to aggressively scrape key-value pairs from broken JSON strings.
    Handles keys with spaces, parentheses, and symbols.
    """
    matches = []

    # Match quoted string pairs: "Any Key" : "Any Value"
    matches.extend(re.findall(r'"([^"]+)"\s*:\s*"([^"]*)"', text))

    # Match unquoted number pairs: "Any Key" : 1234.50
    matches.extend(re.findall(r'"([^"]+)"\s*:\s*([0-9\.\,-]+)(?=[,\}\s])', text))

    # Clean up whitespace and ignore empty keys
    clean_matches = [(k.strip(), v.strip()) for k, v in matches if k.strip()]
    return clean_matches

def evaluate_flat_lists(pred_list, true_list, threshold=0.85):
    """Compares the salvaged predictions against ground truth, handling duplicates gracefully."""
    tp, fp, fn = 0, 0, 0
    pred_pool = pred_list.copy()

    for true_k, true_v in true_list:
        best_match_idx = -1
        best_sim = 0

        # Find the best matching prediction for this specific true key
        for i, (pred_k, pred_v) in enumerate(pred_pool):
            # Relaxed key matching: case-insensitive
            if true_k.lower() == pred_k.lower():
                sim = get_similarity(pred_v, true_v)
                if sim > best_sim:
                    best_sim = sim
                    best_match_idx = i

        # If we found a good fuzzy match, count as True Positive
        if best_sim >= threshold:
            tp += 1
            pred_pool.pop(best_match_idx) # Remove to avoid double counting
        else:
            fn += 1 # We missed this ground truth field

    # Anything left in the prediction pool is considered a hallucinated extra field
    fp = len(pred_pool)
    return tp, fp, fn

# ==========================================
# 2. THE EVALUATION LOOP
# ==========================================

print("\n2. Starting Flat-Entity Bulk Evaluation (Cleaned Ground Truth)...")

task_prompt = "Extract Structured Invoice Data"

total_tp, total_fp, total_fn = 0, 0, 0
perfect_documents = 0

# Note: Assumes `ds_invoice` and `perturb_image` are already defined in your notebook
num_samples = min(50, len(ds_invoice["test"]))

for i in tqdm(range(num_samples), desc="Evaluating Florence-2"):
    sample = ds_invoice["test"][i]

    # Prepare image and ground truth
    clean_image = sample["image"].convert("RGB")
    messy_image = perturb_image(clean_image, mode="blur") # Your heavy blur!

    # Prepare Cleaned Ground Truth as a flat list
    gt_data = sample["ground_truth"]
    true_dict = json.loads(gt_data) if isinstance(gt_data, str) else gt_data
    true_list = flatten_ground_truth(true_dict)

    # Run Inference
    inputs = processor(text=task_prompt, images=messy_image, return_tensors="pt").to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            early_stopping=False,
            do_sample=False,
            num_beams=3,
        )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    final_output_string = generated_text.replace(task_prompt, "").strip()

    # Salvage Data using the relaxed Regex
    pred_list = salvage_predictions(final_output_string)

    # Evaluate the flat lists
    tp, fp, fn = evaluate_flat_lists(pred_list, true_list)

    total_tp += tp
    total_fp += fp
    total_fn += fn

    # Check if the document was perfectly parsed
    if fn == 0 and fp == 0 and tp == len(true_list) and len(true_list) > 0:
        perfect_documents += 1

# ==========================================
# 3. CALCULATING FINAL RESEARCH METRICS
# ==========================================

precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

field_accuracy = total_tp / (total_tp + total_fp + total_fn) if (total_tp + total_fp + total_fn) > 0 else 0
document_accuracy = perfect_documents / num_samples if num_samples > 0 else 0

print("\n" + "="*50)
print("📊 FINAL FLAT-ENTITY METRICS (Florence-2)")
print("="*50)
print(f"Total Samples Tested : {num_samples}")
print("-" * 50)
print(f"Precision            : {precision:.4f}")
print(f"Recall               : {recall:.4f}")
print(f"F1 Score             : {f1_score:.4f}")
print("-" * 50)
print(f"Total Field Accuracy : {field_accuracy:.4f}")
print(f"Document Accuracy    : {document_accuracy:.4f}")
print("="*50)

1. Loading the Custom Brain...

2. Starting Flat-Entity Bulk Evaluation (Cleaned Ground Truth)...


Evaluating Florence-2: 100%|██████████| 50/50 [01:10<00:00,  1.41s/it]


📊 FINAL FLAT-ENTITY METRICS (Florence-2)
Total Samples Tested : 50
--------------------------------------------------
Precision            : 0.6159
Recall               : 0.2773
F1 Score             : 0.3824
--------------------------------------------------
Total Field Accuracy : 0.2364
Document Accuracy    : 0.0000


* Conclusion:Evaluation on heavily perturbed invoices reveals that the fine-tuned Florence-2 model adopts a conservative extraction strategy. Rather than hallucinating incorrect values when blinded by severe blur, the model safely drops unreadable fields—resulting in a low recall of 0.2773, but successfully preserving a strong precision of 0.6159 for the data it can successfully resolve.

**Qwen2-VL-7B Local LLm**

In [ ]:
!pip install accelerate bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 14.7 MB/s eta 0:00:00


In [ ]:
import torch
import json
import re
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# ==========================================
# 1. LOAD QWEN-2.5-7B IN 4-BIT
# ==========================================
print("Loading Qwen-2.5-7B-Instruct (4-bit)...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# device_map="auto" automatically manages your GPU memory
qwen_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)
qwen_model.eval()

# ==========================================
# 2. PIPELINE FUNCTIONS
# ==========================================
def serialize_ocr_for_llm(ocr_words):
    """Converts PaddleOCR spatial output into a readable string format for Qwen."""
    # Sort top-to-bottom, then left-to-right (using a y-tolerance of 10 pixels for rows)
    sorted_words = sorted(ocr_words, key=lambda w: (w['y'] // 10, w['x']))

    text_representation = ""
    for w in sorted_words:
        # Pass the spatial coordinates and the text to the LLM
        text_representation += f"[x:{w['x']}, y:{w['y']}] {w['text']}\n"
    return text_representation

def extract_json_with_qwen(ocr_words):
    """Passes serialized OCR to Qwen and forces a strict JSON output."""
    if not ocr_words:
        return {"menu": [], "total": {"total_price": ""}}

    ocr_text = serialize_ocr_for_llm(ocr_words)

    # Strict Prompt Engineering
    system_prompt = """You are an expert Data Extraction AI.
You are given raw OCR text from a receipt along with their [x, y] pixel coordinates.
Reconstruct the menu items and find the final total price.
You MUST respond with ONLY raw, valid JSON. Do not include markdown formatting like ```json.
Schema:
{
  "menu": [
    {"cnt": "quantity", "nm": "item name", "price": "price"}
  ],
  "total": {
    "total_price": "total amount"
  }
}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Extract data from this OCR output:\n{ocr_text}"}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = qwen_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1, # Low temp = highly factual, less creative hallucination
            do_sample=False
        )

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Clean the output (LLMs sometimes ignore the "no markdown" rule)
    clean_response = response.strip()
    if clean_response.startswith("```json"):
        clean_response = clean_response.split("```json")[1]
    if clean_response.startswith("```"):
        clean_response = clean_response.split("```")[1]
    if clean_response.endswith("```"):
        clean_response = clean_response.rsplit("```", 1)[0]
    clean_response = clean_response.strip()

    try:
        return json.loads(clean_response)
    except json.JSONDecodeError as e:
        # Fallback if the LLM output is structurally broken
        return {"menu": [], "total": {"total_price": ""}}

# ==========================================
# 3. EVALUATION LOOP
# ==========================================
def evaluate_phase1_pipeline(ds, samples=50):
    print(f"\nStarting Phase 1 Evaluation (PaddleOCR -> Qwen-2.5-7B) on {samples} samples...")

    total_correct = 0
    item_count_correct = 0

    for i in tqdm(range(samples), desc="Processing Invoices"):
        sample = ds["train"][i]
        image = sample["image"]

        # Ground Truth
        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        # 1. Run PaddleOCR (using your existing function)
        words = paddle_ocr_structured(image)

        # 2. Run Qwen Extraction
        pred_parse = extract_json_with_qwen(words)

        # Metrics: Total Accuracy
        gt_total = str(gt_parse.get("total", {}).get("total_price", "")).strip()
        pred_total = str(pred_parse.get("total", {}).get("total_price", "")).strip()
        if gt_total == pred_total:
            total_correct += 1

        # Metrics: Item Count Accuracy
        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))
        if gt_items == pred_items:
            item_count_correct += 1

    print("\n" + "="*40)
    print("📊 PHASE 1 PIPELINE RESULTS")
    print("="*40)
    print(f"Total Accuracy %       : {(total_correct/samples)*100:.2f}%")
    print(f"Item Count Accuracy %  : {(item_count_correct/samples)*100:.2f}%")
    print("="*40)

# RUN THE EVALUATION
evaluate_phase1_pipeline(ds_invoice, samples=50)

Loading Qwen-2.5-7B-Instruct (4-bit)...


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]


Starting Phase 1 Evaluation (PaddleOCR -> Qwen-2.5-7B) on 50 samples...


Processing Invoices:   0%|          | 0/50 [00:00<?, ?it/s]

[2026/02/27 12:32:35] ppocr DEBUG: dt_boxes num : 80, elapsed : 1.8796188831329346
[2026/02/27 12:32:36] ppocr DEBUG: cls num  : 80, elapsed : 0.32678914070129395
[2026/02/27 12:32:36] ppocr DEBUG: rec_res num  : 80, elapsed : 0.4437994956970215


...

[2026/02/27 12:42:27] ppocr DEBUG: dt_boxes num : 28, elapsed : 0.04979729652404785
[2026/02/27 12:42:27] ppocr DEBUG: cls num  : 28, elapsed : 0.03986048698425293
[2026/02/27 12:42:27] ppocr DEBUG: rec_res num  : 28, elapsed : 0.08628630638122559


Processing Invoices:  98%|█████████▊| 49/50 [10:07<00:11, 11.60s/it]

[2026/02/27 12:42:41] ppocr DEBUG: dt_boxes num : 9, elapsed : 0.02891063690185547
[2026/02/27 12:42:41] ppocr DEBUG: cls num  : 9, elapsed : 0.01590871810913086
[2026/02/27 12:42:41] ppocr DEBUG: rec_res num  : 9, elapsed : 0.030823707580566406


Processing Invoices: 100%|██████████| 50/50 [10:13<00:00, 12.28s/it]


📊 PHASE 1 PIPELINE RESULTS
Total Accuracy %       : 34.00%
Item Count Accuracy %  : 46.00%


Text-only LLMs are great at parsing the meaning of OCR output, but they are bottlenecked by the OCR engine's mistakes and the loss of visual layout context.

In [ ]:
!pip install qwen-vl-utils accelerate bitsandbytes transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 18.0 MB/s eta 0:00:00


In [ ]:
import torch
import json
from tqdm import tqdm
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# ==========================================
# 1. LOAD QWEN2-VL-7B IN 4-BIT
# ==========================================
print("Loading Qwen2-VL-7B-Instruct (4-bit)...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "Qwen/Qwen2-VL-7B-Instruct"

# Load the VLM
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)
processor = AutoProcessor.from_pretrained(model_id)
model.eval()

# ==========================================
# 2. END-TO-END INFERENCE FUNCTION
# ==========================================
def extract_json_with_vl(image):
    """Passes the raw image directly to the VLM to get JSON."""
    if image.mode != "RGB":
        image = image.convert("RGB")

    system_prompt = """You are an expert Data Extraction AI. Extract the menu items and total price from this receipt image.
Output MUST be strictly valid JSON matching this schema:
{
  "menu": [
    {"cnt": "quantity", "nm": "item name", "price": "price"}
  ],
  "total": {
    "total_price": "total amount"
  }
}
Do not include markdown formatting like ```json or any conversational text."""

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                    # We cap resolution slightly to prevent Kaggle T4 OOM on very long receipts
                    "max_pixels": 800 * 800
                },
                {"type": "text", "text": system_prompt},
            ]
        }
    ]

    # Process inputs for the VLM
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate Output
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=False
        )

    # Trim the prompt from the output
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

    # Clean up standard LLM markdown hallucinations
    clean_response = response.strip()
    if clean_response.startswith("```json"):
        clean_response = clean_response.split("```json")[1]
    if clean_response.startswith("```"):
        clean_response = clean_response.split("```")[1]
    if clean_response.endswith("```"):
        clean_response = clean_response.rsplit("```", 1)[0]
    clean_response = clean_response.strip()

    try:
        return json.loads(clean_response)
    except json.JSONDecodeError:
        return {"menu": [], "total": {"total_price": ""}}

# ==========================================
# 3. EVALUATION LOOP
# ==========================================
def evaluate_phase2_vl(ds, samples=50):
    print(f"\nStarting Phase 2 Evaluation (Direct VLM) on {samples} samples...")

    total_correct = 0
    item_count_correct = 0

    for i in tqdm(range(samples), desc="Processing Invoices"):
        sample = ds["train"][i]
        image = sample["image"]

        # Ground Truth
        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        # Predict directly from image (No PaddleOCR!)
        pred_parse = extract_json_with_vl(image)

        # Metrics
        gt_total = str(gt_parse.get("total", {}).get("total_price", "")).strip()
        pred_total = str(pred_parse.get("total", {}).get("total_price", "")).strip()
        if gt_total == pred_total:
            total_correct += 1

        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))
        if gt_items == pred_items:
            item_count_correct += 1

    print("\n" + "="*40)
    print("📊 PHASE 2 VLM RESULTS (Qwen2-VL)")
    print("="*40)
    print(f"Total Accuracy %       : {(total_correct/samples)*100:.2f}%")
    print(f"Item Count Accuracy %  : {(item_count_correct/samples)*100:.2f}%")
    print("="*40)

# (Assuming ds_invoice is already loaded in your environment)
from datasets import load_dataset
ds_invoice = load_dataset("naver-clova-ix/cord-v2")

evaluate_phase2_vl(ds_invoice, samples=50)

Loading Qwen2-VL-7B-Instruct (4-bit)...


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/730 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


Starting Phase 2 Evaluation (Direct VLM) on 50 samples...



Processing Invoices: 100%|██████████| 50/50 [11:46<00:00, 14.12s/it]


📊 PHASE 2 VLM RESULTS (Qwen2-VL)
Total Accuracy %       : 58.00%
Item Count Accuracy %  : 44.00%


* preprocessing at infrence Images

In [ ]:
import cv2
import numpy as np
from PIL import Image

def optimize_image_for_extraction(image, target="vlm"):
    """
    Cleans a real-world document image before passing it to AI.
    target="ocr" applies aggressive binarization.
    target="vlm" applies gentle enhancement to preserve visual features.
    """
    # 1. Convert PIL to OpenCV format (Grayscale)
    img = np.array(image)
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img

    # 2. Denoise (Removes camera grain)
    denoised = cv2.fastNlMeansDenoising(gray, h=15)

    # 3. Contrast Enhancement (Makes faded text readable)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(denoised)

    if target == "ocr":
        # 4. Aggressive Binarization (Best for PaddleOCR)
        # Forces pixels to pure black or pure white, ignoring shadows
        final_img = cv2.adaptiveThreshold(
            enhanced, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 15, 5
        )
        # Convert back to RGB for your existing functions
        final_img = cv2.cvtColor(final_img, cv2.COLOR_GRAY2RGB)
    else:
        # VLM path (Best for Qwen2-VL)
        # VLMs need to see faint lines, logos, and box borders.
        # Binarization destroys those, so we stop at Contrast Enhancement.
        final_img = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2RGB)

    return Image.fromarray(final_img)

In [ ]:
import torch
import json
from tqdm import tqdm
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# ==========================================
# 1. LOAD QWEN2-VL-7B IN 4-BIT
# ==========================================
print("Loading Qwen2-VL-7B-Instruct (4-bit)...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "Qwen/Qwen2-VL-7B-Instruct"

# Load the VLM
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)
processor = AutoProcessor.from_pretrained(model_id)
model.eval()

# ==========================================
# 2. END-TO-END INFERENCE FUNCTION
# ==========================================
def extract_json_with_vl(image):
    """Passes the image directly to the VLM to get JSON."""

    system_prompt = """You are an expert Data Extraction AI. Extract the menu items and total price from this receipt image.
Output MUST be strictly valid JSON matching this schema:
{
  "menu": [
    {"cnt": "quantity", "nm": "item name", "price": "price"}
  ],
  "total": {
    "total_price": "total amount"
  }
}
Do not include markdown formatting like ```json or any conversational text."""

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                    # Cap resolution to prevent Kaggle T4 OOM
                    "max_pixels": 800 * 800
                },
                {"type": "text", "text": system_prompt},
            ]
        }
    ]

    # Process inputs for the VLM
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate Output
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=False
        )

    # Trim the prompt from the output
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

    # Clean up standard LLM markdown hallucinations
    clean_response = response.strip()
    if clean_response.startswith("```json"):
        clean_response = clean_response.split("```json")[1]
    if clean_response.startswith("```"):
        clean_response = clean_response.split("```")[1]
    if clean_response.endswith("```"):
        clean_response = clean_response.rsplit("```", 1)[0]
    clean_response = clean_response.strip()

    try:
        return json.loads(clean_response)
    except json.JSONDecodeError:
        return {"menu": [], "total": {"total_price": ""}}

# ==========================================
# 3. EVALUATION LOOP WITH PREPROCESSING
# ==========================================
def evaluate_phase2_vl_with_preprocessing(ds, samples=50):
    print(f"\nStarting Phase 2 Evaluation (With Preprocessing) on {samples} samples...")

    total_correct = 0
    item_count_correct = 0

    for i in tqdm(range(samples), desc="Processing Invoices"):
        sample = ds["train"][i]
        raw_image = sample["image"]

        # Ground Truth
        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        # --------------------------------------------------
        # CALL YOUR PREPROCESSING FUNCTION HERE
        # We use target="vlm" to preserve layout features!
        # --------------------------------------------------
        clean_image = optimize_image_for_extraction(raw_image, target="vlm")

        # Predict directly from the CLEANED image
        pred_parse = extract_json_with_vl(clean_image)

        # Metrics
        gt_total = str(gt_parse.get("total", {}).get("total_price", "")).strip()
        pred_total = str(pred_parse.get("total", {}).get("total_price", "")).strip()
        if gt_total == pred_total:
            total_correct += 1

        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))
        if gt_items == pred_items:
            item_count_correct += 1

    print("\n" + "="*45)
    print("📊 PHASE 2 RESULTS (With Preprocessing)")
    print("="*45)
    print(f"Total Accuracy %       : {(total_correct/samples)*100:.2f}%")
    print(f"Item Count Accuracy %  : {(item_count_correct/samples)*100:.2f}%")
    print("="*45)

# Run the evaluation!
evaluate_phase2_vl_with_preprocessing(ds_invoice, samples=50)

To push this prototype from 60% to enterprise-grade accuracy, you now have the exact blueprint for your production deployment:

Remove the Resolution Cap (The Hardware Fix): Right now, we are compressing massive, vertical receipts into an 800x800 square to avoid crashing the free Kaggle T4 GPU. In production, deploying Qwen2-VL-7B on a larger GPU (like an NVIDIA A10G with 24GB VRAM) allows you to pass the image at 2000x2000 resolution. Once the VLM can actually read the fine print, the Item Count Accuracy will surge.

Implement Constrained Decoding (The Software Fix): Instead of writing Python scripts to clean up markdown blocks or hoping the VLM doesn't miss a comma, the production pipeline will wrap the model in a library like Outlines or vLLM. This forces the VLM at the hardware level to only generate tokens that perfectly match your JSON schema.

**Applying Prompting and strict Json for better accuracy**

In [ ]:
!pip install lm-format-enforcer pydantic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.0 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from typing import List

# Define exactly what a menu item should look like
class MenuItem(BaseModel):
    cnt: str
    nm: str
    price: str

# Define the total section
class InvoiceTotal(BaseModel):
    total_price: str

# Define the final JSON structure
class InvoiceExtraction(BaseModel):
    menu: List[MenuItem]
    total: InvoiceTotal

In [ ]:
import torch
import json
from tqdm import tqdm
# ==========================================
# 1. THE DEPENDENCY BUG FIX (Monkey Patch)
# ==========================================
# This fixes the HuggingFace 'transformers' file location change
# so lm-format-enforcer can import successfully without crashing.
import transformers.tokenization_utils_base
import transformers.tokenization_utils
transformers.tokenization_utils.PreTrainedTokenizerBase = transformers.tokenization_utils_base.PreTrainedTokenizerBase
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn

def extract_json_with_vl_constrained(image):
    """Passes the image to the VLM and FORCES strict JSON output."""
    if image.mode != "RGB":
        image = image.convert("RGB")
    # 1. Setup the parser based on our Pydantic schema
    try:
        schema_dict = InvoiceExtraction.model_json_schema()
    except AttributeError:
        schema_dict = InvoiceExtraction.schema()

    parser = JsonSchemaParser(schema_dict)
    prefix_function = build_transformers_prefix_allowed_tokens_fn(
        processor.tokenizer, parser
    )

    system_prompt = "Extract the menu items and total price from this receipt image."

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image, "max_pixels": 800 * 800},
                {"type": "text", "text": system_prompt},
            ]
        }
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # 2. Generate Output with the ENFORCER attached
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=False,
            # This single line forces the model to follow the JSON rules!
            prefix_allowed_tokens_fn=prefix_function
        )

    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

    # 3. Because it is constrained, it is GUARANTEED to be valid JSON.
    # No cleanup required.
    return json.loads(response)

In [ ]:
from pydantic import BaseModel
from typing import List
import torch
import json
from tqdm import tqdm

# ==========================================
# 1. THE DEPENDENCY BUG FIX (Monkey Patch)
#    (Copied from cell -NH7lbtARI2l to ensure availability)
# ==========================================
import transformers.tokenization_utils_base
import transformers.tokenization_utils
transformers.tokenization_utils.PreTrainedTokenizerBase = transformers.tokenization_utils_base.PreTrainedTokenizerBase
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn

# ==========================================
# 2. Pydantic Models
#    (Copied from cell T-mMP7jNO0uY to ensure availability)
# ==========================================
# Define exactly what a menu item should look like
class MenuItem(BaseModel):
    cnt: str
    nm: str
    price: str

# Define the total section
class InvoiceTotal(BaseModel):
    total_price: str

# Define the final JSON structure
class InvoiceExtraction(BaseModel):
    menu: List[MenuItem]
    total: InvoiceTotal

# ==========================================
# 3. Model and Processor Loading
#    (Copied and adapted from cell vjc82tGqlDA6 for robustness)
# ==========================================
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

print("Ensuring Qwen2-VL-7B-Instruct (4-bit) model and processor are loaded...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "Qwen/Qwen2-VL-7B-Instruct"

# Check if model and processor are already defined globally
# This avoids re-loading heavy models if they are already in memory
try:
    # Attempt to access an attribute to confirm existence and functionality
    # Check if 'model' is in globals before accessing it to avoid KeyError
    if 'model' in globals():
        _ = globals()['model'].config
    else:
        raise KeyError("'model' not found in globals")

    if 'processor' in globals():
        _ = globals()['processor'].tokenizer
    else:
        raise KeyError("'processor' not found in globals")

    print("Model and processor already loaded globally.")
except (NameError, AttributeError, KeyError): # Added KeyError to the exception list
    print("Model or processor not found or invalid, loading them now...")
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config
    )
    processor = AutoProcessor.from_pretrained(model_id)
model.eval()

# ==========================================
# 4. Modified `extract_json_with_vl_constrained` function
#    (Copied from cell -NH7lbtARI2l and modified to accept processor and model)
# ==========================================
def extract_json_with_vl_constrained(image, current_processor, current_model):
    """Passes the image to the VLM and FORCES strict JSON output."""
    if image.mode != "RGB":
        image = image.convert("RGB")
    # 1. Setup the parser based on our Pydantic schema
    try:
        schema_dict = InvoiceExtraction.model_json_schema()
    except AttributeError:
        schema_dict = InvoiceExtraction.schema()

    parser = JsonSchemaParser(schema_dict)
    prefix_function = build_transformers_prefix_allowed_tokens_fn(
        current_processor.tokenizer, parser
    )

    system_prompt = "Extract the menu items and total price from this receipt image."

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image, "max_pixels": 800 * 800},
                {"type": "text", "text": system_prompt},
            ]
        }
    ]

    text = current_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = current_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # 2. Generate Output with the ENFORCER attached
    with torch.no_grad():
        generated_ids = current_model.generate(
            **inputs,
            max_new_tokens=1536,
            temperature=0.1,
            do_sample=False,
            # This single line forces the model to follow the JSON rules!
            prefix_allowed_tokens_fn=prefix_function
        )

    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = current_processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

    # <-- ADDED SAFETY NET: Prevents the loop from crashing if a receipt is insanely long
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        print("\n[Warning] A receipt was too long and hit the token limit. Skipping...")
        return {"menu": [], "total": {"total_price": ""}}

# ==========================================
# 5. `evaluate_phase2_vl_with_preprocessing` function
#    (Original content of the selected cell, modified to pass processor and model)
# ==========================================
def evaluate_phase2_vl_with_preprocessing(ds, samples=20):
    print(f"\nStarting Phase 2 Evaluation (With Preprocessing) on {samples} samples...")

    total_correct = 0
    item_count_correct = 0

    for i in tqdm(range(samples), desc="Processing Invoices"):
        sample = ds["train"][i]
        raw_image = sample["image"]

        # Ground Truth
        gt = json.loads(sample["ground_truth"])
        gt_parse = gt["gt_parse"]

        # --------------------------------------------------
        # CALL YOUR PREPROCESSING FUNCTION HERE
        # We use target="vlm" to preserve layout features!
        # --------------------------------------------------
        clean_image = optimize_image_for_extraction(raw_image, target="vlm")

        # Predict directly from the CLEANED image
        pred_parse = extract_json_with_vl_constrained(clean_image, processor, model)

        # Metrics
        gt_total = str(gt_parse.get("total", {}).get("total_price", "")).strip()
        pred_total = str(pred_parse.get("total", {}).get("total_price", "")).strip()
        if gt_total == pred_total:
            total_correct += 1

        gt_items = len(gt_parse.get("menu", []))
        pred_items = len(pred_parse.get("menu", []))
        if gt_items == pred_items:
            item_count_correct += 1

    print("\n" + "="*45)
    print("📊 PHASE 2 RESULTS (With Preprocessing)")
    print("="*45)
    print(f"Total Accuracy %       : {(total_correct/samples)*100:.2f}%")
    print(f"Item Count Accuracy %  : {(item_count_correct/samples)*100:.2f}%")
    print("="*45)

# Run the evaluation!
evaluate_phase2_vl_with_preprocessing(ds_invoice, samples=20)

Ensuring Qwen2-VL-7B-Instruct (4-bit) model and processor are loaded...
Model and processor already loaded globally.

Starting Phase 2 Evaluation (With Preprocessing) on 20 samples...


Processing Invoices: 100%|██████████| 20/20 [13:56<00:00, 41.83s/it]


📊 PHASE 2 RESULTS (With Preprocessing)
Total Accuracy %       : 65.00%
Item Count Accuracy %  : 60.00%
